# Tool recommendation with Attention network 
## (Gated recurrent units Attention neural network with weighted cross-entropy loss)

In [91]:
import numpy as np
import json
import warnings
import operator

import tensorflow as tf
from keras import backend as K
import h5py

warnings.filterwarnings("ignore")


class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units, **kwargs):
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
        self.units = units
        super(BahdanauAttention, self).__init__(**kwargs)

    def get_config(self):
        return super().get_config().copy()

    def call(self, query, values):
        # query hidden state shape == (batch_size, hidden size)
        # query_with_time_axis shape == (batch_size, 1, hidden size)
        # values shape == (batch_size, max_len, hidden size)
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights


class ToolPredictionAttentionModel():
  
    def __init__(self, parameters):
        self.embedding_size = int(parameters["embedding_size"])
        self.gru_units = int(parameters["units"])
        self.max_len = parameters["max_len"]
        self.dimensions = parameters["dimensions"]
        self.learning_rate = parameters["learning_rate"]
        self.class_weights = parameters["class_weights"]
        self.spatial_dropout = parameters["spatial_dropout"]
        self.recurrent_dropout = parameters["recurrent_dropout"]
        self.dropout = parameters["dropout"]
        
    def weighted_loss(self, class_weights):
        """
        Create a weighted loss function. Penalise the misclassification
        of classes more with the higher usage
        """
        weight_values = list(class_weights.values())

        def weighted_binary_crossentropy(y_true, y_pred):
            # add another dimension to compute dot product
            expanded_weights = tf.keras.backend.expand_dims(weight_values, axis=-1)
            return tf.keras.backend.dot(tf.keras.backend.binary_crossentropy(y_true, y_pred), expanded_weights)
        return weighted_binary_crossentropy

    def create_model(self):
        sequence_input = tf.keras.layers.Input(shape=(self.max_len,), dtype='int32')
        embedded_sequences = tf.keras.layers.Embedding(self.dimensions, self.embedding_size, input_length=self.max_len, mask_zero=True)(sequence_input)
        embedded_sequences = tf.keras.layers.SpatialDropout1D(self.spatial_dropout)(embedded_sequences)
        gru_1 = tf.keras.layers.GRU(self.gru_units,
            return_sequences=True,
            return_state=False,
            activation='elu',
            recurrent_dropout=self.recurrent_dropout
        )
        gru_2 = tf.keras.layers.GRU(self.gru_units,
            return_sequences=True,
            return_state=True,
            activation='elu',
            recurrent_dropout=self.recurrent_dropout
        )
        gru_output = gru_1(embedded_sequences)
        gru_output = tf.keras.layers.Dropout(self.dropout)(gru_output)
        gru_output, gru_hidden = gru_2(gru_output)
        attention = BahdanauAttention(self.gru_units)
        context_vector, attention_weights = attention(gru_hidden, gru_output)
        dropout = tf.keras.layers.Dropout(self.dropout)(context_vector)
        output = tf.keras.layers.Dense(self.dimensions, activation='sigmoid')(dropout)
        model = tf.keras.Model(inputs=sequence_input, outputs=output)
        model.compile(
            optimizer=tf.keras.optimizers.RMSprop(learning_rate=self.learning_rate),
            loss=self.weighted_loss(self.class_weights),
        )
        return model


def load_model(model_path):
    trained_model = h5py.File(model_path, 'r')
    dictionary = json.loads(trained_model.get('data_dictionary').value)
    best_parameters = json.loads(trained_model.get('parameters').value)
    compatible_tools = json.loads(trained_model.get('compatible_tools').value)
    reverse_dictionary = dict((str(v), k) for k, v in dictionary.items())
    new_model = ToolPredictionAttentionModel(best_parameters).create_model()
    new_model.load_weights(model_path)
    return dictionary, reverse_dictionary, class_weights, best_parameters, compatible_tools, new_model


model_path = "data/tool_recommendation_attention_model.hdf5"
dictionary, reverse_dictionary, class_weights, best_parameters, compatible_tools, new_model = load_model(model_path)

## Unpack trained model for prediction

In [92]:
print(reverse_dictionary)

{'1': 'gops_intersect_1', '2': 'coords2clnt.py', '3': 'blockbuster', '4': 'Add_a_column1', '5': 'blockclust', '6': 'Datamash', '7': 'methtools_calling', '8': 'methtools_dmr', '9': 'ctb_simsearch', '10': 'trim_galore', '11': 'deeptools_correctGCBias', '12': 'sed_stream_editor', '13': 'rgPicardMarkDups', '14': 'fasta_filter_by_length', '15': 'seq_filter_by_id', '16': 'deeptools_bamCoverage', '17': 'piranha', '18': 'vcftools_merge', '19': 'ssake', '20': 'cuffcompare', '21': 'flexbar_split_RYYR_bcs', '22': 'tp_head_tool', '23': 'r_correlation_matrix', '24': 'wc_gnu', '25': 'cshl_awk_replace_in_column', '26': 'ncbi_blastp_wrapper', '27': 'stringtie', '28': 'mergeCols1', '29': 'cshl_fastq_quality_filter', '30': 'Show tail1', '31': 'CONVERTER_interval_to_bedstrict_0', '32': 'term_id_vs_term_name', '33': 'aragorn_trna', '34': 'filter_bed_on_splice_junctions', '35': 'subtract_query1', '36': 'fastq_groomer', '37': 'cuffdiff', '38': 'tp_sed_tool', '39': 'Grouping1', '40': 'convert_bc_to_binary_RY

In [93]:
def compute_recommendations(tool_sequence, labels, topk=20, max_seq_len=25):
    tl_seq = tool_sequence.split(",")
    last_tool_name = reverse_dictionary[str(tl_seq[-1])]
    sample = np.zeros(max_seq_len)
    for idx, tool_id in enumerate(tl_seq):
        sample[idx] = int(tool_id)
    sample_reshaped = np.reshape(sample, (1, max_seq_len))
    tool_sequence_names = [reverse_dictionary[str(tool_pos)] for tool_pos in tool_sequence.split(",")]
    # predict next tools for a test path
    prediction = new_model.predict(sample_reshaped, verbose=0)
    class_weighs = best_parameters["class_weights"]
    weight_val = list(class_weights.values())
    weight_val = np.reshape(weight_val, (len(weight_val),))
    prediction = np.reshape(prediction, (prediction.shape[1],))
    #prediction = prediction * weight_val
    prediction = prediction / float(np.max(prediction))
    prediction_pos = np.argsort(prediction, axis=-1)
    # get topk prediction
    topk_prediction_pos = prediction_pos[-topk:]
    topk_prediction_pos = [item for item in topk_prediction_pos if item != 0]
    topk_prediction_val = [int(prediction[pos] * 100) for pos in topk_prediction_pos]
    # read tool names using reverse dictionary
    pred_tool_ids = [reverse_dictionary[str(tool_pos)] for tool_pos in topk_prediction_pos]
    pred_tool_ids_sorted = dict()
    for (tool_pos, tool_pred_val) in zip(topk_prediction_pos, topk_prediction_val):
        tool_name = reverse_dictionary[str(tool_pos)]
        pred_tool_ids_sorted[tool_name] = tool_pred_val
    pred_tool_ids_sorted = dict(sorted(pred_tool_ids_sorted.items(), key=lambda kv: kv[1], reverse=True))
    ids_tools = dict()
    keys = list(pred_tool_ids_sorted.keys())
    tool_seq_name = ",".join(tool_sequence_names)
    print("Current tool sequence: ")
    print()
    print(tool_seq_name)
    print()
    print("Recommended tools for the tool sequence '%s' with their scores in decreasing order:" % tool_seq_name)
    print()
    for i in pred_tool_ids_sorted:
        print(i + "(" + str(pred_tool_ids_sorted[i]) + "%)")
    for key in pred_tool_ids_sorted:
        ids_tools[key] = dictionary[key]
    print()
    print("Tool ids:")
    print()
    for i in ids_tools:
        print(i + "(" + str(ids_tools[i]) + ")")

## Indices of tools

In [94]:
topk = 20 # set the maximum number of recommendations
tool_seq = "2" # give tools ids in a sequence and see the recommendations. To know all the tool ids, 
                     # please print the variable 'reverse_dictionary'
compute_recommendations(tool_seq, "", topk)

Current tool sequence: 

coords2clnt.py

Recommended tools for the tool sequence 'coords2clnt.py' with their scores in decreasing order:

modencode_peakcalling_macs2(100%)
scaffold2fasta(99%)
deeptools_computeGCBias(99%)
Show tail1(99%)
deeptools_correctGCBias(99%)
openms_id_mapper(99%)
deeptools_computeMatrix(99%)
deeptools_bamFingerprint(99%)
tp_awk_tool(99%)
flexbardsc(99%)
gatk2_variant_apply_recalibration(98%)
openms_id_file_converter(98%)
computeMatrix(98%)
bedtools_coveragebed_counts(98%)
rmcontamination(98%)
get_flanks1(98%)
bgchem_fragment_merger(98%)
bedtools_intersectBed(98%)
cshl_cut_tool(98%)

Tool ids:

modencode_peakcalling_macs2(226)
scaffold2fasta(227)
deeptools_computeGCBias(183)
Show tail1(30)
deeptools_correctGCBias(11)
openms_id_mapper(191)
deeptools_computeMatrix(198)
deeptools_bamFingerprint(42)
tp_awk_tool(72)
flexbardsc(181)
gatk2_variant_apply_recalibration(142)
openms_id_file_converter(77)
computeMatrix(205)
bedtools_coveragebed_counts(263)
rmcontamination(16

## Recommended tools

In [95]:
print(new_model.summary())

Model: "model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_21 (InputLayer)        [(None, 25)]              0         
_________________________________________________________________
embedding_20 (Embedding)     (None, 25, 49)            15190     
_________________________________________________________________
spatial_dropout1d_8 (Spatial (None, 25, 49)            0         
_________________________________________________________________
gru_21 (GRU)                 (None, 25, 96)            42336     
_________________________________________________________________
dropout_6 (Dropout)          (None, 25, 96)            0         
_________________________________________________________________
gru_22 (GRU)                 [(None, 25, 96), (None, 9 55872     
_________________________________________________________________
bahdanau_attention_16 (Bahda ((None, 96), (None, 25, 1 187

In [89]:
print(best_parameters)

{'batch_size': 51.0, 'dropout': 0.22602806806492431, 'embedding_size': 33.0, 'learning_rate': 0.0032540638980664565, 'recurrent_dropout': 0.4216428563134349, 'spatial_dropout': 0.07267624883456103, 'units': 85.0, 'max_len': 25, 'dimensions': 310, 'class_weights': {'0': 0.0, '1': 1.2679033691330028, '2': 0.0953052070646881, '3': 2.36302973908641, '4': 0.3291822433584368, '5': 0.5994148283307562, '6': 0.0, '7': 4.893910221874146, '8': 0.38600688549788165, '9': 1.9296225475826647, '10': 5.249888185668796, '11': 1.2584609896100056, '12': 5.5060777871737265, '13': 0.2527023535557542, '14': 0.09531017980432493, '15': 0.09531017980432493, '16': 0.38668123051024755, '17': 4.116595171156921, '18': 7.360040357734192, '19': 3.6149637711637683, '20': 0.09531017980432493, '21': 0.1879510232386989, '22': 5.78484241624876, '23': 0.0, '24': 0.09543005443725577, '25': 7.13966033596492, '26': 0.09473615144520271, '27': 2.7111174575911785, '28': 0.0, '29': 2.1856234256723175, '30': 0.02803959057123678, '